# Information about Ice raids in February, 2017

Interesting, there is problem with converting the .xlsx file. I had to open it in Excel and `save as` first before csvkit would properly process the contents of the file.


In [1]:
%%bash
# Convert to csv
in2csv data-feb-raw/reinstated.xlsx > data-feb/reinstated.csv

/Users/christian/anaconda/envs/data/lib/python3.5/site-packages/openpyxl/workbook/names/named_range.py:124: UserWarning: Discarded range with reserved name
  warnings.warn("Discarded range with reserved name")
/Users/christian/anaconda/envs/data/lib/python3.5/site-packages/openpyxl/workbook/workbook.py:102: UserWarning: Call to deprecated function or class get_active_sheet (Use the .active property).
  def get_active_sheet(self):


In [2]:
%%bash
# this trims stuff from the head and tail
count=$(wc -l < data-feb/reinstated.csv | sed 's/ //g'); trim=$(echo $count - 10 | bc); \
tail -n +6 data-feb/reinstated.csv | head -n $trim > data-feb/reinstated-trimmed.csv


In [3]:
%%bash
# report line count, stripping spaces
wc -l < data-feb/reinstated-trimmed.csv | sed 's/ //g'


201


In [4]:
import csv
import datetime

In [5]:
# helper functions

# This is the name unmangler
def name_unmangler(name_str):
    """Return `LAST,REST` as a tuple"""
    try:
        name_split = name_str.split(',')
        return (name_split[0], name_split[1])
    except:
        return name_str


In [48]:
# scrape spreadsheet data into a python dictionary
with open('data-feb/reinstated-trimmed.csv', 'r') as infile:
    data = csv.reader(infile, delimiter=',')
    data = list(data)
    
    # setting up our people dataframe
    people = []
    
    # setting up our charges dataframe
    charges = []
    
    booking_id_tracker = ''
    
    # set initial defaults    
    new_record = False
    booking_id = None

    # We are working through the data in the csv
    for row in data:
        row = [x.strip() for x in row]

        if new_record:
            booking_id = row[0]
            booking_id_tracker = booking_id #sets the tracker to this record
            name = row[1]
            name_last = name_unmangler(name)[0]
            name_rest = name_unmangler(name)[1]
            race = row[2]
            sex = row[3]
            jail_custody = row[4]
            age = row[5]
            booking_date = row[6]
            release_date = row[7]
            nativity = row[8]

            people.append([
                    booking_id,
                    name_last,
                    name_rest,
                    race,
                    sex,
                    jail_custody,
                    age,
                    booking_date,
                    release_date,
                    nativity
                ])
        
            # reset flag
            new_record = False
        
        else:
            if ''.join(row).strip() != '' and row[1].strip() != 'Charge' and row[0] != 'Booking No':
                print(booking_id_tracker, row)
        
        # setting the flag at the end of the loop so we know it is
        # a new record.
        # if it's a blank row, the next one is a new record
        if ''.join(row).strip() == '' or row[0] == 'Booking No':
            # set flag
            new_record = True



1541823 ['NEWART', '36010001', 'INDECENCY W/CHILD SEXUAL CONTACT', '', 'F2', '', '70000', '', '']
1541823 ['ICEW', '4200', 'ICE DETAINER', '', 'F*', '', '', '', '']
1601778 ['NEW', '48010019', 'EVADING ARREST DET W/PREV CONVICTION', '', 'FS', '', '', '2016-01-15', 'NO CHARGES FILED']
1601778 ['NEW', '48010020', 'EVADING ARREST DET W/VEH', '', 'F3', '7 years', '', '', '']
1601778 ['NEW', '54040011', 'DRIVING WHILE INTOXICATED 3RD OR MORE', '', 'F3', '7 years', '', '', '']
1601778 ['NEW', '29990042', 'CRIMINAL MISCHIEF >=$100<$750', '', 'MB', '', '', '2016-01-14', 'NO CHARGES FILED']
1601778 ['NEW', '35990014', 'POSS CS PG1<1G (FS)', '', 'FS', '', '', '2016-01-14', 'NO CHARGES FILED']
1601778 ['NEW', '29990027', 'CRIM MISCHIEF IMPAIR/INTERRUPT PUB SERVICE', '', 'FS', '', '', '', '']
1601778 ['ICEW', '4200', 'ICE DETAINER', '', 'F*', '', '', '', '']
1601778 ['REIN', '54040011', 'DRIVING WHILE INTOXICATED 3RD OR MORE', '', 'F3', '', '', '2017-01-18', 'DISMISSAL']
1621712 ['NEW', '54040011'